In [39]:
pip install pandas transformers


In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
import csv
import re
import string
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

df_ytb = pd.read_csv("/content/drive/MyDrive/CSV FILES/ytb__Getting myself together_.csv")

# Remove column name 'Unnamed: 0'
dfc = df_ytb.drop(['Unnamed: 0','username'], axis=1)
# dfc = df_ytb.drop(index=11)
dfc = dfc.reset_index(drop=True)

#list_of_sentences = dfc['comment'].tolist()
#dfc
def clean_html(text):
    clean=re.compile('<.*?>')
    cleantext=re.sub(clean,'',text)
    return cleantext

# first round of cleaning
def clean_text1(text):
    text=text.lower()
    text=re.sub('','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\w*\d\w*','',text)
    return text

# second round of cleaning
def clean_text2(text):
    text=re.sub('[''"",,,]','',text)
    text=re.sub('\n','',text)
    return text

cleaned_html=lambda x:clean_html(x)
cleaned1=lambda x:clean_text1(x)
cleaned2=lambda x:clean_text2(x)

dfc['review'] = dfc['comment']
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned_html))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned1))
dfc['review']=pd.DataFrame(dfc.review.apply(cleaned2))
print(dfc.shape)
dfc.head(10)

first10comments = dfc.review.values

sentences = first10comments
list_of_sentences = dfc['comment'].tolist()


(30, 2)


In [42]:
tokenizer = AutoTokenizer.from_pretrained("aiknowyou/it-emotion-analyzer")
model = AutoModelForSequenceClassification.from_pretrained("aiknowyou/it-emotion-analyzer")

emotion_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
emotion_analysis(list_of_sentences)
results = emotion_analysis(list_of_sentences)
results

[{'label': '2', 'score': 0.9614323973655701},
 {'label': '2', 'score': 0.4866494834423065},
 {'label': '2', 'score': 0.46415066719055176},
 {'label': '1', 'score': 0.8597398996353149},
 {'label': '1', 'score': 0.42199406027793884},
 {'label': '2', 'score': 0.7542903423309326},
 {'label': '1', 'score': 0.5334919095039368},
 {'label': '2', 'score': 0.8793052434921265},
 {'label': '2', 'score': 0.6269951462745667},
 {'label': '1', 'score': 0.4819933772087097},
 {'label': '2', 'score': 0.8881772756576538},
 {'label': '2', 'score': 0.8787174820899963},
 {'label': '2', 'score': 0.6516135931015015},
 {'label': '2', 'score': 0.6966861486434937},
 {'label': '2', 'score': 0.8437832593917847},
 {'label': '2', 'score': 0.8002176284790039},
 {'label': '2', 'score': 0.6011666655540466},
 {'label': '1', 'score': 0.501500129699707},
 {'label': '2', 'score': 0.8346793055534363},
 {'label': '2', 'score': 0.7100287079811096},
 {'label': '2', 'score': 0.604054868221283},
 {'label': '2', 'score': 0.5224566

In [43]:
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
# Map numerical labels to emotions in the results
dfc['Emotion'] = [emotion_mapping.get(result['label'], result['label']) for result in results]
dfc['Score'] = [result['score'] for result in results]

final =dfc[['comment', 'Emotion', 'Score']]
final


,comment,Emotion,Score
0,This movie clearly shows that if u do what you...,2,0.961432
1,In this world there are very few things that a...,2,0.486649
2,This scene means &quot;I&#39;m remaking myself...,2,0.464151
3,I&#39;m finally focusing on myself and working...,1,0.859740
4,This is a video about overcoming procrastination,1,0.421994
5,Everyone in a moment of our life we have a sum...,2,0.754290
6,my favorite scene,1,0.533492
7,"Value love more than anything , don&#39;t take...",2,0.879305
8,"In the end, Tom realized he still has the tale...",2,0.626995
9,People say re-watch your favourite movie or re...,1,0.481993


In [44]:
# Create a new column 'mapped_emotion' using emotion_mapping
emotion_mapping = {
    0: 'sadness',
    1: 'joy',
    2: 'love',
    3: 'anger',
    4: 'fear',
    5: 'surprise'
}
final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')

final['Mapped_Emotion'] = final['Emotion'].map(emotion_mapping)

# Display the updated 'final' DataFrame with the new column
print(final[['Emotion', 'Mapped_Emotion']])

    Emotion Mapped_Emotion
0         2           love
1         2           love
2         2           love
3         1            joy
4         1            joy
5         2           love
6         1            joy
7         2           love
8         2           love
9         1            joy
10        2           love
11        2           love
12        2           love
13        2           love
14        2           love
15        2           love
16        2           love
17        1            joy
18        2           love
19        2           love
20        2           love
21        2           love
22        1            joy
23        2           love
24        2           love
25        1            joy
26        2           love
27        3          anger
28        2           love
29        4           fear


<ipython-input-44-a9bc13b09b5c>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['Emotion'] = pd.to_numeric(final['Emotion'], errors='coerce')


In [45]:
# Add sentiment analysis results to the DataFrame
# Save the DataFrame with sentiment results to a new CSV file
output_csv_path = "/content/drive/MyDrive/ytb_Getting myself together.csv"
final.to_csv(output_csv_path, index=False)

# Display the first few rows of the DataFrame with sentiment results
final

,comment,Emotion,Score,Mapped_Emotion
0,This movie clearly shows that if u do what you...,2,0.961432,love
1,In this world there are very few things that a...,2,0.486649,love
2,This scene means &quot;I&#39;m remaking myself...,2,0.464151,love
3,I&#39;m finally focusing on myself and working...,1,0.859740,joy
4,This is a video about overcoming procrastination,1,0.421994,joy
5,Everyone in a moment of our life we have a sum...,2,0.754290,love
6,my favorite scene,1,0.533492,joy
7,"Value love more than anything , don&#39;t take...",2,0.879305,love
8,"In the end, Tom realized he still has the tale...",2,0.626995,love
9,People say re-watch your favourite movie or re...,1,0.481993,joy
